## Introdução

Nesta atividade iremos melhorar nosso sistema de recuperação de informação por meio de expansão de consultas. A expansão de consultas é uma técnina na qual se acrescenta termos na consulta original relacionados aos termos já existntes para que os documentos recuperados sejam mais específicos e úteis.

Os dados utilizados nesta tarefa são sobre notícias e estão organizados em um arquivo csv que possui as seguintes colunas:

timestamp, titulo, subTitulo, conteúdo, url, idNoticia

Os nomes das colunas são bastante claros e autodescritivos.

Na tabela abaixo podemos ver uma pequena porção dos dados.

In [1]:
import pandas as pd

news_df = pd.read_csv("data/estadao_noticias_eleicao.csv", engine = "python")

# Substitui os valores faltantes (NaN) por strings vazias
news_df.fillna(" ", inplace = True)

news_df.head()

,timestamp,titulo,subTitulo,conteudo,url,idNoticia
0,2014-12-31T00:00:00Z,PT espera 30 mil pessoas em festa na Esplanada,Objetivo é demonstrar apoio popular a Dilma e ...,BRASÍLIA - Após o desgaste provocado com o lan...,"http://politica.estadao.com.br/noticias/geral,...",1
1,2014-12-31T00:00:00Z,Alckmin toma posse de olho no Planalto,Governador reeleito tenta amarrar tucanos paul...,"Reeleito em outubro, o governador tucano Geral...","http://politica.estadao.com.br/noticias/geral,...",2
2,2014-12-31T00:00:00Z,Seis obstáculos e desafios do segundo mandato ...,"Em meio a escândalo de corrupção, presidente t...",1. Rearranjo das contas A nova equipe econôm...,"http://politica.estadao.com.br/noticias/geral,...",3
3,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",4
4,2014-12-31T00:00:00Z,,Veja as principais fotos do dia e dos eventos ...,,"http://fotos.estadao.com.br/fotos/politica,dil...",5


## Trantando os dados

Para otimizar nossas consultar, primeiro vamos realizar algumas operações sobre os dados. Vamos tokenizar os documentos e remover palavras muito comuns assim como palavras que só aparecem uma única vez no corpus inteiro.

In [2]:
content = news_df.titulo + " " + news_df.subTitulo + " " + news_df.conteudo

# Remove palavras muito comuns, remove pontuação e tokeniza o corpus
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer(r'\w+')

stoplist = set('a o e de do da com em na no nas nos as os dos das para por'.split())
texts = [[word for word in tokenizer.tokenize(news.lower()) if word not in stoplist]
         for news in content]

# Remove palavras que aprecem uma única vez
from collections import defaultdict

frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] > 1]
         for text in texts]


## Passo 1: Escrever uma função que retorne uma matrix de termos-termos contendo as frequências de co-ocorrência de duas palavras consecutivas.

Supondo o seguinte documento:

"A B C B A B A" 

A matrix de termos-termos seria:

|   | A | B | C |
|---|---|---|---|
| A | 0 | 2 | 0 |
| B | 2 | 0 | 1 |
| C | 0 | 1 | 0 |

Toda vez que um termo X ocorrer imediatamente após um termo Y, o número na linha X, coluna Y deverá ser incrementado em 1.

Abaixo o cógido para função que cria um modelo dessa matrix de co-ocorrências utilizando a função Word2Vec da biblioteca gensim. A função Word2Vec cria um modelo de vocabulário do corpus que entre outras coisas servirá como matriz de co-ocorrências para nós

In [3]:
from gensim.models import Word2Vec

# Cria um modelo de vocabulário do corpus
model = Word2Vec(texts, size=100, window=5, min_count=1, workers=4)

print(model)

Word2Vec(vocab=38304, size=100, alpha=0.025)


## Passo 2: Escrever uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorneas top-3 palavras em ordem decrescente de frequencia.

Abaixo encontra-se o código da função

In [4]:
# Retorna uma lista com as três palavras mais similares à palavra passada como argumento
def top_n_query(term, model):
    return [i[0] for i in model.wv.most_similar(positive=[term], topn=3)]

## Passos 3 e 4: Expandir a consulta original com os termos retornados no passo 2 acima; Fazer uma busca disjuntiva (OR) considerando a nova consulta.


Iremos agora expandir nossa consulta de documentos no corpus utilizando também as top-3 palavras mais próximas da palavra passada como argumento da consulta.

Primeiro precisamos criar o índece invertido e criar a função de busca nesse índice. Como isto já foi feito em tarefas passadas irei apenas reaproveitar o código.

In [5]:
"""Transforma uma string, convertendo-a para caixa baixa e removendo a pontuação."""
def preprocess(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = text.lower()
    return tokenizer.tokenize(text)

"""Cria uma estrutura de índice invertido com as informações de 
Frequência de Termo (TF) e Frequência de Documento Invertida (IDF).

argumentos:
df -- Um dataframe pandas 
"""
def create_indexer(df):
    global indexer
    indexer = {}

    for index, row in df.iterrows():
        document = row['titulo'] + " " + row['subTitulo'] + " " + row['conteudo']
        document_words = preprocess(document)
        doc_id = row['idNoticia']

        for term in document_words:
            if term in indexer:
                posting = indexer[term]
                if (doc_id in posting):
                    TF = indexer[term][doc_id] + 1
                    indexer[term][doc_id] = TF
                else:
                    indexer[term][doc_id] = 1
            else:
                indexer[term] = {doc_id: 1}
                
    indexer["__all_docs_count__"] = df.shape[0]
                
                
# Cria o índece invertido, armazenando-o na variável global 'indexer'
create_indexer(news_df)

Abaixo encontra-se a função de busca que ranqueia os documentos utilizando a técnica BM25. No caso da expansão de consultas, ele irá criar um ranking para cada termo pesquisado e retornar o ranking final com os documentos mais bem pontuados. Caso um mesmo documento apareça em buscas de diferentes termos, o score que permanence é o maior dentre as buscas.

In [6]:
import math

# HELPERS:
"""
Conta a frequência das palavras em uma lista de palavras.

argumentos:
list_of_words -- A lista de palavras.
"""
def word_freq(list_of_words):
    result = {}
    
    for word in list_of_words:  # Conta a frequência das palavras na consulta
        if word in result:
            result[word] = result[word] + 1 
        else:
            result[word] = 1
    
    return result

"""
Calcula o BM25 de um termo em um documento no índice.

argumentos:
term_freq -- A quantidade de vezes que o termo aparece no documento.
k - O parâmetro de ajuste do algorítmo.
"""
def BM25(term_freq, k = 5): 
    return (((k + 1) * term_freq) / (term_freq + k))

"""
Calcula o IDF (Inverse Document Frequency) de um termo no índice.

argumentos:
total_docs -- O número total de documentos no índice.
doc_frequency -- A quantidade de documentos que contêm o termo.
"""
def IDF(total_docs, doc_frequency):
    return math.log((total_docs + 1) / doc_frequency)

"""
Atualiza o dicionário de documentos mantendo sempre os resultados com maior BM25
"""
def update_results(results, expansion):
    for key, value in expansion.items():
        if key not in results or (key in results and value > results[key]):
            results[key] = value
    return results

"""
Realiza o ranqueamento dos documentos de acordo com a técnica BM25
"""
def BM25_VSM(query):
    query_words = preprocess(query)
    
    query_terms = word_freq(query_words)
    
    docs = {}  # dicionário que guarda os resultados <doc_id: pontuação-no-ranking>
    
    total_docs = indexer["__all_docs_count__"]
    
    for term in query_terms:  # Calcula o ranking para cada documento utilizando TF
        query_freq = query_terms[term]   # a frequência da palavra na consulta
        posting = indexer[term]          # documentos com aquele termo
        docs_freq = len(posting)         # a frequencia do termo em todos os documentos
        idf = IDF(total_docs, docs_freq)
        for doc_id in posting:
            term_freq = posting[doc_id]  # frequencia do termo no documento
            bm25 = BM25(term_freq)
            score_for_term = query_freq * bm25 * idf # Calcula a pontuação do documento para certo termo
            if doc_id in docs:             
                docs[doc_id] = docs[doc_id] + score_for_term 
            else:
                docs[doc_id] = score_for_term
                
    return docs

"""
Efetua uma busca normal, sem expansão e retorna os top 10 documentos
"""
def normal_search(term):
    docs = BM25_VSM(term)
    ranked_docs = sorted(docs, key = docs.get, reverse = True)[:9]
    
    return ranked_docs

"""
Efetua uma busca expandida com os termos mais similares e retorna os top 10 documentos
"""
def expanded_search(term):
    similar_words = top_n_query(term, model)
    
    query = similar_words + [term]
    
    docs = {}
    for term in query:
        docs = update_results(docs, BM25_VSM(term))
        
    ranked_docs = sorted(docs, key = docs.get, reverse = True)[:9]
    
    return ranked_docs     
    

## Avaliação

Escolha livremente três termos de consulta e responda o seguinte:

#### Quais os termos retornados para a expansão de cada consulta?

Os termos escolhidos foram: Corrupção, eleições e deputados.

In [7]:
print(top_n_query("corrupção", model))
print(top_n_query("eleições", model))
print(top_n_query("deputados", model))

['escândalos', 'escândalo', 'denúncias']
['urnas', 'eleição', 'presidenciais']
['vereadores', 'cadeiras', 'senadores']


/home/luiz/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


Como podemos ver acima, as três palavras mais similares para cada um dos termos foram:

* corrupção: escândalo, escândalos e denúncias
* eleições: urnas, eleição e presidenciais
* deputados: vereadores, cadeiras e senadoress

#### Você acha que esses termos são de fato relacionados com a consulta original? Justifique

Acho que sim. Para mim, tdas as palavras retornadas estão relacionadas com as palavras da consulta.

#### Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?

Vamos comparar as pesquisas normais e expandidas de cada termo. Os resultados dos top 10 documentos podem ser vistos nas tabelas abaixo (não estão ordenados pelo ranking).

* Para corrupção:


In [14]:
normal = normal_search("corrupção")
expanded = expanded_search("corrupção")

normal_df = news_df[news_df['idNoticia'].isin(normal)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]
expanded_df = news_df[news_df['idNoticia'].isin(expanded)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]

normal_df

/home/luiz/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,titulo,subTitulo,conteudo,idNoticia
47,"Corrupção, escândalo e produção legislativa","Rogério Fernando Taffarello*\n\nMais uma vez, ...","Rogério Fernando Taffarello* Mais uma vez, com...",48
154,Justiça abre primeira ação contra cartel das e...,Executivos da Engevix Engenharia são acusados ...,"Por Fausto Macedo e Ricardo Brandt, enviado es...",155
204,"Em discurso, Janot fala em 'gestão desastrosa'...",Na Conferência Internacional de Combate à Corr...,Brasília - Em um duro discurso na manhã desta ...,205
474,Corrupção exige que governo se posicione e ten...,"Aceitemos então, de uma vez por todas, a voz d...","Aceitemos então, de uma vez por todas, a voz d...",475
711,A reforma política seria o principal para o co...,Por Ana Carla Bliacheriene*,Tenho cá minhas dúvidas. O discurso oficial re...,712
6567,“Leis redundantes e superadas inviabilizam pun...,Antônio d'Ávila avalia que 'estruturas burocrá...,Antônio d’Ávila avalia que ‘estruturas burocrá...,6568
7016,Juízes defendem nova Lei Orgânica da Justiça F...,Lei do período ditatorial é criticada por magi...,Lei do período ditatorial é criticada por magi...,7017
7339,Frente cobra Congresso por não combater corrup...,Em seminário que reuniu políticos e jornalista...,A Frente Parlamentar contra a Corrupção acusou...,7340
7834,"“Os juízes podem muito, mas não podem tudo”","Para juiz federal, a corrupção está enraizada ...","Para juiz federal, a corrupção está enraizada ...",7835


In [15]:
expanded_df

,titulo,subTitulo,conteudo,idNoticia
47,"Corrupção, escândalo e produção legislativa","Rogério Fernando Taffarello*\n\nMais uma vez, ...","Rogério Fernando Taffarello* Mais uma vez, com...",48
162,Veja a lista de 36 denunciados pela Procurador...,Acusações envolvem recursos desviados da ordem...,"Por Ricardo Brandt, enviado especial a Curitib...",163
204,"Em discurso, Janot fala em 'gestão desastrosa'...",Na Conferência Internacional de Combate à Corr...,Brasília - Em um duro discurso na manhã desta ...,205
222,‘País convulsiona com o maior escândalo de cor...,"Em mensagem a seus pares, procurador geral da ...","Por Fausto Macedo, Ricardo Brandt e Julia Affo...",223
711,A reforma política seria o principal para o co...,Por Ana Carla Bliacheriene*,Tenho cá minhas dúvidas. O discurso oficial re...,712
1659,Dilma aponta divergência entre parte do PSB e ...,,A petista alegou que a base do PSB mais ligada...,1660
1730,Cantor popular engajado,Fagner e Beth Carvalho resgatam tempo em que a...,"No início, estavam todos juntos e misturados....",1731
2046,Surpresas e dilemas tornaram escolha difícil,"Morte de Eduardo Campos, escândalos de corrupç...",O bombardeio do horário eleitoral e do noticiá...,2047
6465,Ministro do Supremo diz que País vive ‘apagão ...,"Em São Paulo, Gilmar Mendes condena 'dimensão ...","Em São Paulo, Gilmar Mendes condena ‘dimensão ...",6466


* Para eleições:

In [16]:
normal = normal_search("eleições")
expanded = expanded_search("eleições")

normal_df = news_df[news_df['idNoticia'].isin(normal)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]
expanded_df = news_df[news_df['idNoticia'].isin(expanded)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]

normal_df

/home/luiz/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,titulo,subTitulo,conteudo,idNoticia
784,Veja a íntegra da resolução aprovada pela Exec...,Documento defende maior participação da legend...,Na primeira reunião após a reeleição da presid...,785
832,"Para Janot, pedido do PSDB de auditoria das el...","Em parecer, procurador-geral da República crit...",Atualizado às 21h55 Brasília - O procurador-ge...,833
1839,O voto dos pobres sempre causou polêmica,Ataques nas redes sociais não questionam a int...,Houve certo exagero na interpretação dada à fa...,1840
2141,Imprensa internacional destaca indefinição nas...,"Segundo o espanhol 'El País', jovens, protagon...",SÃO PAULO - Alguns jornais e sites de notícias...,2142
2614,Vivemos eleições inconstitucionais?,"Em abril deste ano, o STF iniciou o julgamento...","Em abril deste ano, o STF iniciou o julgamento...",2615
2888,Votação em prisões aumenta 19% nas eleições de...,O número de presos provisórios e adolescentes ...,Por Julia Affonso O número de presos provisóri...,2889
4830,Limite de gastos na disputa presidencial de 20...,"Há duas décadas, na 1ª campanha em que empresa...",Os 11 candidatos que vão concorrer à Presidênc...,4831
5128,Procurador fecha cerco aos fichas sujas em São...,"André de Carvalho Ramos, fiscal do maior colég...",Fausto Macedo e Mateus Coutinho O procurador r...,5129
6484,Três municípios de Goiás e RN elegem novos pre...,,"Nestes casos, segundo a Resolução nº 23.280/20...",6485


In [17]:
expanded_df

,titulo,subTitulo,conteudo,idNoticia
424,Governo tem a iniciativa,\n\nO governo parece ter logrado êxito na tar...,O governo parece ter logrado êxito na tarefa...,425
590,"Em Brasília, 400 vão às ruas por impeachment d...",,O PT precisa entender que o Brasil não é deles...,591
784,Veja a íntegra da resolução aprovada pela Exec...,Documento defende maior participação da legend...,Na primeira reunião após a reeleição da presid...,785
865,'Estamos longe da modernidade',Eleição revelou País que sai da cordialidade p...,"O Brasil que sai das urnas é o Brasil real, em...",866
1206,Refém do qualunquismo?,,"O termo vem do italiano “qualunque”, que signi...",1207
2068,Skaf agradece votação e deseja sucesso ao eleito,Candidato do PMDB ao governo de São Paulo teve...,SÃO PAULO - O candidato do PMDB ao governo est...,2069
2197,"Em Pernambuco, Monteiro minimiza revés nas pes...",,"De acordo com o levantamento, Câmara, herdeiro...",2198
4830,Limite de gastos na disputa presidencial de 20...,"Há duas décadas, na 1ª campanha em que empresa...",Os 11 candidatos que vão concorrer à Presidênc...,4831
5128,Procurador fecha cerco aos fichas sujas em São...,"André de Carvalho Ramos, fiscal do maior colég...",Fausto Macedo e Mateus Coutinho O procurador r...,5129


* Para deputados:

In [18]:
normal = normal_search("deputados")
expanded = expanded_search("deputados")

normal_df = news_df[news_df['idNoticia'].isin(normal)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]
expanded_df = news_df[news_df['idNoticia'].isin(expanded)][['titulo', 'subTitulo', 'conteudo', 'idNoticia']]

normal_df

/home/luiz/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,titulo,subTitulo,conteudo,idNoticia
6,Veja os desafios dos governadores que assumem ...,,"No Acre, governador reeleito quer erradicar an...",7
715,As 10 empresas que mais doaram em 2014 ajudam ...,Financiadores colaboraram com 360 das 513 camp...,São Paulo - Sete de cada dez deputados federai...,716
758,Eleição do Congresso 2015 custa R$ 1 bilhão,Campanha mais cara da história teve arrecadaçã...,"São Paulo - Não há crise econômica, pelo menos...",759
855,Partidos que surgiram após governo Collor terã...,"Congresso. Pela primeira vez, legendas criadas...",A Câmara dos Deputados que toma posse em 2015 ...,856
870,Número de políticos estreantes eleitos para As...,Apenas 15% dos deputados estaduais vitoriosos ...,Dos 1.057 parlamentares eleitos para as 26 Ass...,871
1412,PT perde para PSDB posto de partido com maior ...,"Petistas receberam 1,75 milhão de votos na leg...",Brasília - O PT perdeu nas eleições deste ano ...,1413
1782,Câmara nova recebe 63% de 30 empresas,Estudo calcula concentração de doações obtidas...,Três dezenas de grupos empresariais doaram 63%...,1783
1941,"Base de Dilma elege 304 deputados, mas alinham...",Avaliação é de que resultado das eleições pres...,Os nove partidos que apoiam formalmente a reel...,1942
7190,Deputados estaduais viram alvo de investigação,Procuradores pedem ao TJ autorização para ouvi...,A Procuradoria-Geral de Justiça abriu investig...,7191


In [19]:
expanded_df

,titulo,subTitulo,conteudo,idNoticia
758,Eleição do Congresso 2015 custa R$ 1 bilhão,Campanha mais cara da história teve arrecadaçã...,"São Paulo - Não há crise econômica, pelo menos...",759
1051,Presidente continua a ter maioria no Senado,Aliados de Dilma ocupam 53 das 81 cadeiras da ...,"Brasília - Ao menos no papel, a presidente Dil...",1052
2020,PMDB mantém hegemonia no Senado,Partido elege 5 senadores e mantém maior bancada,BRASÍLIA - Ao conquistar no último domingo 5 d...,2021
4211,A tentativa da ‘volta dos que se foram’ em Sabino,A cidade de Sabino fica a mais de 400km da cap...,A cidade de Sabino fica a mais de 400km da cap...,4212
4795,Pimentel recebe vereadores de Cláudio e provoc...,Candidato do PT ao governo de Minas cobra pres...,Belo Horizonte - O candidato do PT ao governo ...,4796
6028,Renan alega que TSE interferiu em matéria do L...,,O recurso ao Supremo é mais um round na queda ...,6029
6181,"Robson Marinho dá graças a Jeca Tatu, sem o qu...",por Fernando Gallo\n\nNão fosse Jeca Tatu e o ...,por Fernando Gallo Não fosse Jeca Tatu e o con...,6182
7226,O Município no Congresso Nacional,Por Eder Brito\n\nNão se deixe enganar pelo tí...,Por Eder Brito Não se deixe enganar pelo títul...,7227
7483,Câmara cassa mandato do prefeito de Campo Grande,"Segundo os vereadores, Alcides Bernal (PP) com...",A Câmara de Campo Grande (MS) cassou na noite ...,7484


Olhando para os resultados obtidos, vemos que tanto na consulta normal quanto na consulta expandida os documentos retornados estão bem próximos do que era desejado. Acredito que isso ocorra pois a nossa coleção de documentos tem um tema bem específico - notícias políticas. Acredito que se a coleção fosse mais geral sobre diversos temas, etnão teríamos uma diferença nos resultados das diferentes buscas. Isso também ocorre porquê muitas das palavras mais similares costumam aparecer nos documentos em que a palavra da consulta original aparece, por isso as vezes aparecem os mesmos documentos tanto no resultado das consultas normais quanto nos resultados das consultas expandidas.

#### A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?


Na definição de precision, sabe-se que este é a fração de documentos relevantes entre os documentos recuperados e recall é a fração de documentos relevantes recuperados entre o total de documentos relevantes.

Dessa forma, ao expandir a busca o número de documentos que dão match na consulta aumenta e consequentemente o número de documentos relevantes. Dessa forma a precisão diminui, já que na forma em que estas são calculadas, aumentar uma implica na diminuição da outra.